# BA optimization

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import selfies as sf
import torch
import time
from tqdm import tqdm, trange
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [ ]:
## Inter-op parallelism
torch.set_num_interop_threads(4)
torch.get_num_interop_threads()
## Intra-op parallelism
torch.set_num_threads(4)
torch.get_num_threads()

In [ ]:
class GPUCONFIGS:
    def __init__(self):
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device('cuda:0' if self.use_cuda else 'cpu')
        if self.use_cuda: torch.cuda.set_device(self.device)
        
gpuconfigs = GPUCONFIGS()
print(torch.cuda.current_device())

In [ ]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.stackVAE import StackAugmentedVAE
from rebadd.datautils import GeneratorData
from rebadd.reinforce import REINFORCE_SCST_OFFPOLICY as REINFORCE

# 1. Generator

## 1-1) Data loading for generators

In [ ]:
class DATACONFIGS:
    def __init__(self):
        ## input
        self.input_dir = os.path.join('processed_data', 'gsk3_jnk3_qed_sa')
        self.train_data_path = os.path.join(self.input_dir, 'fragments_list.pkl')
        self.vocab_data_path = os.path.join(self.input_dir, 'vocabulary.csv')
        ## output
        self.output_dir = os.path.join('outputs_2_optimize_ReBADD', 'gsk3_jnk3_qed_sa')
        assert os.path.exists(self.output_dir)

dataconfigs = DATACONFIGS()

In [ ]:
gen_data = GeneratorData(pickle_data_path=dataconfigs.train_data_path,
                         vocabulary_path=dataconfigs.vocab_data_path,
                         use_cuda=gpuconfigs.use_cuda)

print(f"Number of training samples: {len(gen_data.data)}")
print(f"Number of vocabulary: {len(gen_data.vocabs)}")
print(f"Maximum of seqlen: {gen_data.max_seqlen}")

## 1-2) Pretrained generators

In [ ]:
kwargs_generator = {"input_size"         : gen_data.n_characters,
                    "output_size"        : gen_data.n_characters,
                    "max_seqlen"         : 40,
                    "hidden_size"        : 256,
                    "latent_size"        : 64,
                    "n_layers"           : 4,
                    "has_stack"          : True,
                    "stack_width"        : 256,
                    "stack_depth"        : 20,
                    "lr"                 : 1e-4,
                    "use_cuda"           : gpuconfigs.use_cuda,
                    "device"             : gpuconfigs.device,
                    "optimizer_instance" : torch.optim.RMSprop}

In [ ]:
my_generator_after  = StackAugmentedVAE(**kwargs_generator)

In [ ]:
class CKPTCONFIGS:
    def __init__(self):
        self.model_path = os.path.join('outputs_1_pretraining_ReBADD', 'gsk3_jnk3_qed_sa', 'checkpoint.pth')
        
ckptconfigs = CKPTCONFIGS()

In [ ]:
my_generator_after.load_model(ckptconfigs.model_path)

# 2. Reinforcement learning

In [ ]:
from ReBADD_config import Reward_gsk3_jnk3_qed_sa as Reward

## 2-1) RL design

In [ ]:
my_reward_ft = Reward()

In [ ]:
ReBADD = REINFORCE(data=gen_data, generator=my_generator_after, reward_ft=my_reward_ft, zeta=0.9999)

## 2-2) Training

In [ ]:
model_path = os.path.join(dataconfigs.output_dir, 'checkpoint.pth')
losses_path = os.path.join(dataconfigs.output_dir, 'losses.csv')
plot_path = os.path.join(dataconfigs.output_dir, "plots_reward_losses.pdf")

In [ ]:
def SMILES_generate(generator, n_to_generate, gen_data):
    generated = []
    for i in trange(n_to_generate):
        z = generator.sample_latent_vectors()
        sel = generator.evaluate(gen_data, z=z)
        sel = sel.replace(gen_data.start_token, "").replace(gen_data.end_token, "")
        smi = sf.decoder(sel)
        generated.append(smi)
    return generated

In [ ]:
def simple_moving_average(previous_values, new_value, ma_window_size=10):
    value_ma = np.sum(previous_values[-(ma_window_size-1):]) + new_value
    value_ma = value_ma/(len(previous_values[-(ma_window_size-1):]) + 1)
    return value_ma

In [ ]:
def show_reward_loss(rewards, losses, figsize=(8,2.5), filepath=None):
    fig, ax = plt.subplots(1,2,figsize=figsize)
    
    ax[0].plot(rewards)
    ax[0].set_xlabel('Training iteration')
    ax[0].set_ylabel('Average reward')
    ax[1].plot(rl_losses, 'r')
    ax[1].set_xlabel('Training iteration')
    ax[1].set_ylabel('Loss')
    
    plt.tight_layout()
    if filepath is None:
        plt.show()
    else:
        plt.savefig(filepath)

In [ ]:
def write_history(tr_rewards, rl_losses, reconst_losses, entropy_losses,
                  gsk_list, jnk_list, qed_list, sa_list,
                  execution_times, filepath):
    
    df_history = pd.DataFrame({"REWARD":tr_rewards,
                               "LOSS":rl_losses,
                               "LOSS_RECONST":reconst_losses,
                               "LOSS_ENTROPY":entropy_losses,
                               "GSK3":gsk_list,
                               "JNK3":jnk_list,
                               "QED":qed_list,
                               "SA":sa_list,
                               "EXEC_TIME":execution_times})
    
    df_history.to_csv(filepath, index=False)
    return df_history

In [ ]:
# Setting up some parameters for the experiment
n_iterations = 1000
checkpoint_stepsize = 50
behavior_stepsize = 20

In [ ]:
tr_rewards = [] # rewards when training the model
rl_losses = [] # loss when training the model
reconst_losses = [] # reconstruction loss
entropy_losses = [] # entropy regularization

gsk_list = []
jnk_list = []
qed_list = []
sa_list = []

execution_times = []

## Training RL optimizer
start_time = time.time()

alpha_min = 0.01
alpha_max = 0.1
alpha_inc = (alpha_max - alpha_min) / n_iterations
alpha = alpha_min
for i in range(1, n_iterations+1):
    ## policy gradient
    alpha += alpha_inc
    cur_reward, cur_loss, cur_reconst, cur_entropy = ReBADD.policy_gradient(grad_clipping=1., alpha=alpha)
    
    ## append - reward
    reward = cur_reward
    tr_rewards.append(reward)
    
    ## append - loss
    rl_loss = cur_loss
    rl_losses.append(rl_loss)
    reconst_loss = cur_reconst
    reconst_losses.append(reconst_loss)
    entropy_loss = cur_entropy
    entropy_losses.append(entropy_loss)
        
    ## execution time check
    end_time = time.time()
    exec_time = end_time - start_time
    execution_times.append(exec_time)
    
    ## print - logs
    print(f"[{i:04d}] rl_loss: {rl_loss:.3f}\treconst_loss: {reconst_loss:.3f}\tentropy_loss: {entropy_loss:.3f}\treward: {reward:.3f}\ttau: {ReBADD.tau:.3f}\t({exec_time:.3f} sec)")
        
    ## checkpoint
    avg_gsk = 0.
    avg_jnk = 0.
    avg_qed = 0.
    avg_sa = 0.
    
    if i % checkpoint_stepsize == 0:
        ## save
        ReBADD.generator.save_model(model_path.replace('.pth', f'.pth.{i:04d}'))
        
        ## plotting
        show_reward_loss(tr_rewards, rl_losses)

        ## sampling check
        smiles_cur = SMILES_generate(ReBADD, n_to_generate=100, gen_data=gen_data)
        print(f'Sample trajectories: {len(smiles_cur)}')
        if len(smiles_cur) > 0:
            for sm in smiles_cur:
                _, s_gsk, s_jnk, s_qed, s_sa = my_reward_ft(sm, debug=True)
                print(f"[{s_gsk:.3f}, {s_jnk:.3f}, {s_qed:.3f}, {s_sa:.3f}]    {sm}")
                avg_gsk += s_gsk
                avg_jnk += s_jnk
                avg_qed += s_qed
                avg_sa += s_sa
            avg_gsk /= len(smiles_cur)
            avg_jnk /= len(smiles_cur)
            avg_qed /= len(smiles_cur)
            avg_sa /= len(smiles_cur)
    
    ## append - binding affinity values
    gsk_list.append(avg_gsk)
    jnk_list.append(avg_jnk)
    qed_list.append(avg_qed)
    sa_list.append(avg_sa)
    
    ## history save
    _ = write_history(tr_rewards, rl_losses, reconst_losses, entropy_losses,
                      gsk_list, jnk_list, qed_list, sa_list,
                      execution_times, losses_path)
    
    if i % behavior_stepsize == 0:
        ReBADD.update_behavior()

# 3. Save

In [ ]:
ReBADD.generator.save_model(model_path)

In [ ]:
show_reward_loss(tr_rewards, rl_losses, filepath=plot_path)

In [ ]:
df_history = write_history(tr_rewards, rl_losses, reconst_losses, entropy_losses,
                           gsk_list, jnk_list, qed_list, sa_list,
                           execution_times, losses_path)
print(df_history.shape)
print(df_history.head())

In [ ]:
def save_smiles(filepath, smiles):
    with open(filepath, 'w') as fout:
        for smi in smiles:
            fout.write(f"{smi}\n")
    print(f"[INFO] {len(smiles)} SMILES were saved in {filepath}")

In [ ]:
n_sampling = 30000

smi_after = SMILES_generate(ReBADD, n_sampling, gen_data)

In [ ]:
save_smiles(os.path.join(dataconfigs.output_dir, "smi_after.csv"), smi_after)